In [2]:
"""(q6.py) M/G/c queueing system with service time monitors"""

from SimPy.Simulation import *
import random

## Erlang distribution ----------
def erlangvariate(k,mu):
    """random variate from an Erlang(k,mu) distribution"""
    result = 0.0
    for i in range(k):
        result += random.expovariate(mu)
    return result

## Model ----------
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb, k, mu):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(k, mu))
            t = random.expovariate(lamb)
            yield hold, self, t

class Arrival(Process):
    """an arrival"""
    def run(self, k, mu):
        arrivetime = now()
        yield request, self, G.server
        t = erlangvariate(k,mu)
        G.servicemon.observe(t)
        G.servicesquaredmon.observe(t**2)
        yield hold, self, t
        yield release, self, G.server
        delay = now()-arrivetime
        G.delaymon.observe(delay)
        #print(now(), "Observed delay", delay)


class G:
    server = 'dummy'
    delaymon = 'Monitor'
    servicemon = 'Monitor'
    servicesquaredmon = 'Monitor'
    
def model(c, N, lamb, k, mu, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c)
    G.delaymon = Monitor()
    G.servicemon = Monitor()
    G.servicesquaredmon = Monitor()
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb, k, mu))
    simulate(until=maxtime)
   
    # gather performance measures
    W = G.delaymon.mean()
    S = G.servicemon.mean()
    S2 = G.servicesquaredmon.mean()
    
    print("Service times:\n")
    print(G.servicemon.yseries())
    print("Service times squared:\n")
    print(G.servicesquaredmon.yseries())
    return(W,S,S2)

## Experiment ----------------
result = model(c=1, N=30, lamb=3, k=2, mu=10,
               maxtime=2000000, rvseed=123)
print("")
print("Estimate of W:",result[0])
print("Estimate of E[tau]:",result[1])
print("Estimate of E[tau^2]:",result[2])


Service times:

(0.06141925443818422, 0.08072068298398319, 0.23148915446894425, 0.08174670665640571, 0.05748620930082159, 0.09827457314816287, 0.2952873764067431, 0.24422765952063172, 0.11602499967195132, 0.32836066732166724, 0.11705570996467347, 0.2406074168069659, 0.19744399927343295, 0.2171753419351639, 0.28739689656094314, 0.05015977987476573, 0.19160945860647904, 0.1259643866527879, 0.05723699179858774, 0.12609989216034093, 0.10820031212126002, 0.10195168475906521, 0.3684716954455296, 0.4320108532755533, 0.09270383828342971, 0.5410935744664773, 0.035936183755936, 0.07512958135128789, 0.06603288201295375, 0.05032547876320646)
Service times squared:

(0.0037723248157424116, 0.006515828661400713, 0.05358722863674673, 0.006682524049168446, 0.003304664259777867, 0.009657891727453615, 0.08719463466517757, 0.05964714967492561, 0.013461800548876303, 0.10782072784393062, 0.013702039235333757, 0.057891929022521015, 0.038984132849087395, 0.047165129144655364, 0.08259697615286145, 0.002516003